In [1]:
import os
import torch
from training import train_model, hyperparameter_tuning
from model import LSTMEncoderOnly, TransformerEncoderOnly, AdvancedLSTM

In [2]:
file_path = os.getcwd() + "/data" # run from root
X_train, y_train = torch.load(file_path + "/train_sequences.pt", weights_only=True)
X_val, y_val = torch.load(file_path + "/val_sequences.pt", weights_only=True) 

print(f"Train sequences: {X_train.shape}, Targets: {y_train.shape}")

# Hyperparameter tuning
best_params = hyperparameter_tuning(X_train, y_train, X_val, y_val, n_trials=15)

# Full training with best hyperparameters
print("\nTraining final model with best hyperparameters...")
#model = LSTMEncoderOnly(input_dim=26, hidden_dim=128, output_dim=1, num_layers=2, dropout=0.2)
#model = TransformerEncoderOnly(input_dim=41, hidden_dim=128, output_dim=1, num_layers=2, dropout=0.2, nhead=4)
adv_model = AdvancedLSTM(input_dim=26, hidden_dim=best_params['hidden_dim'], 
                            output_dim=1, num_layers=best_params['num_layers'], 
                            dropout=best_params['dropout'], num_fc_layers=best_params['num_fc_layers'])


train_model(
    adv_model,
    X_train=X_train, y_train=y_train,
    X_val=X_val, y_val=y_val, 
    learning_rate=best_params['learning_rate'],
    weight_decay=best_params['weight_decay'],
    epochs=100  # Full training
)

Train sequences: torch.Size([22472, 5, 26]), Targets: torch.Size([22472, 1])
Running random search with 15 trials...

Trial 1/15
Params: {'learning_rate': 0.0005, 'hidden_dim': 96, 'weight_decay': 0.0001, 'num_layers': 1, 'dropout': 0.4, 'num_fc_layers': 2, 'batch_size': 32}


/opt/anaconda3/envs/fpl_helper/lib/python3.11/site-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.4 and num_layers=1
  warnings.warn(
Epoch  1/10:  78%|███████▊  | 549/703 [00:03<00:01, 150.38it/s, loss=23.5535, lr=9.59e-5]


KeyboardInterrupt: 